In [301]:
import os
import nltk
import io

# you'll need to run nltk.download() and download all packages
#nltk.download_shell()
nltk.download('stopwords')
#nltk.data.path.append('D:\\JacobSchool\\CS540\\nltk_data')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matthewgorbett/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [302]:
from nltk.tokenize import sent_tokenize, word_tokenize

Sets from http://www.iaees.org/publications/journals/selforganizology/articles/2016-3(3)/algorithm-to-transform-natural-language-into-SQL-queries.pdf
Escape Word Set<br/>
Expression Mapping Set<br/>
Noun Set<br/>
Verb Set<br/>
Semantic Set<br/>
Variable Set<br/>
Relation Set<br/>
Attribute Set<br/>
Conjunction Set<br/>
This is all the sets for words and rules:

In [303]:
escape_words = set(stopwords.words('english'))#["a", "an", "the", "which", "is", "of", "with", "to", "for", "are", "and", "should", "be"]

rules_on_top_of = []

rules_side_by_side = []

nouns = ["block", "wildcard", "block0", "block1", "block2", "block3", "block4", "block5", "block6", "block7", "block8", "block9" \
         "block10", "block11", "block12", "block13", "block14", "block15", "block16", "block17", "block18", "block19" \
         "block20", "block21", "block22", "block23", "block24", "block25", "block26", "block27", "block28", "block29" \
         "block30", "block31", "block32", "block33", "block34", "block35", "block36", "block37", "block38", "block39" \
         "block40", "block41", "block42", "block43", "block44", "block45", "block46", "block47", "block48", "block49" \
         "block50", "block51", "block52", "block53", "block54", "block55", "block56", "block57", "block58", "block59" \
         "block60", "block61", "block62", "block63", "block64", "block65", "block66", "block67", "block68", "block69" \
         "block70", "block71", "block72", "block73", "block74", "block75", "block76", "block77", "block78", "block79" \
         "block80", "block81", "block82", "block83", "block84", "block85", "block86", "block87", "block88", "block89" \
         "block90", "block91", "block92", "block93", "block94", "block95", "block96", "block97", "block98", "block99" \
         "wildcard0", "wildcard1", "wildcard2", "wildcard3", "wildcard4", "wildcard5", "wildcard6", "wildcard7", "wildcard8", "wildcard9" \
         "wildcard10", "wildcard11", "wildcard12", "wildcard13", "wildcard14", "wildcard15", "wildcard16", "wildcard17", "wildcard18", "wildcard19" \
         "wildcard20", "wildcard21", "wildcard22", "wildcard23", "wildcard24", "wildcard25", "wildcard26", "wildcard27", "wildcard28", "wildcard29" \
         "wildcard30", "wildcard31", "wildcard32", "wildcard33", "wildcard34", "wildcard35", "wildcard36", "wildcard37", "wildcard38", "wildcard39" \
         "wildcard40", "wildcard41", "wildcard42", "wildcard43", "wildcard44", "wildcard45", "wildcard46", "wildcard47", "wildcard48", "wildcard49" \
         "wildcard50", "wildcard51", "wildcard52", "wildcard53", "wildcard54", "wildcard55", "wildcard56", "wildcard57", "wildcard58", "wildcard59" \
         "wildcard60", "wildcard61", "wildcard62", "wildcard63", "wildcard64", "wildcard65", "wildcard66", "wildcard67", "wildcard68", "wildcard69" \
         "wildcard70", "wildcard71", "wildcard72", "wildcard73", "wildcard74", "wildcard75", "wildcard76", "wildcard77", "wildcard78", "wildcard79" \
         "wildcard80", "wildcard81", "wildcard82", "wildcard83", "wildcard84", "wildcard85", "wildcard86", "wildcard87", "wildcard88", "wildcard89" \
         "wildcard90", "wildcard91", "wildcard92", "wildcard93", "wildcard94", "wildcard95", "wildcard96", "wildcard97", "wildcard98", "wildcard99" ]

verbs_top = ["top", "above", "over", "on"]
verbs_below = ["below", "underneath",  "under"]
verbs_side = ["side", "neighboring", "beside", "next"]

variables_colors = ["amber", "amethyst", "apricot", "aquamarine", "azure", "beige", "black", "blue", "blush", "bronze", "brown", "burgundy", "byzantium", "carmine", "cerise", "cerulean", "champagne", "chocolate", "coffee", "copper", "coral", "crimson", "cyan", "emerald", "erin", "gold", "gray", "green", "harlequin", "indigo", "ivory", "jade", "lavender", "lemon", "lilac", "lime", "magenta", "maroon", "mauve", "navy", "ochre", "olive", "orange", "orchid", "peach", "pearl", "periwinkle", "pink", "plum", "puce", "purple", "raspberry", "red", "rose", "ruby", "salmon", "sangria", "sapphire", "scarlet", "silver", "tan", "taupe", "teal", "turquoise", "ultramarine", "violet", "viridian", "white", "yellow"]
variables_location = ["0","zero","1", "one", "2", "two", "3", "three", "4", "four", "5", "five", "6", "six", "7", "seven", "8", "eight", "9", "nine"]

Custom Tags:<br/>
NN &rightarrow; Noun<br/>
VT &rightarrow; Verb Top<br/>
VB &rightarrow; Verb Below<br/>
VS &rightarrow; Verb Side<br/>
VarC &rightarrow; Variable Color<br/>
VarL &rightarrow; Variable Location<br/>

In [304]:
def createBlockWorld(tagged_sentence, outputfile):
    block1=""
    block2=""
    relation=""
    command=""
    location=""
    color=""
    swap_on_top_of = False
    print('tagged sentence')
    print(tagged_sentence)
    #Find the first block, should be the block
    for ts in tagged_sentence:
        if (ts[1] == "VT"):
            relation = "on-top-of"
        
        if (ts[1] == "VS"):
            relation = "side-by-side"
        
        if (ts[1] == "VB"):
            swap_on_top_of = True
            relation = "on-top-of"
            
        if (ts[1] == "VarL"):
            if(location==""): 
                location+=ts[0]
            else:
                location+=(" "+ts[0])
            
        
        if (("block" in ts[0]) or ("wildcard" in ts[0])) and block1 == "":
            block1=ts[0]
        
        if (ts[0] != block1) and (("block" in ts[0]) or ("wildcard" in ts[0])):
                block2=ts[0]
        if (ts[1] == "VarC"):
            color = ts[0]
            

    thecommand = ""
    #Create the command for an is relation
    if block1 and block2 and relation:
        if swap_on_top_of:
            thecommand = "(is %s %s %s)" %(block2, block1, relation)
        else:
            thecommand = "(is %s %s %s)" %(block1, block2, relation)
    
    if block1 and color:
        thecommand = "(has %s color %s)" % (block1,color)

    if block1 and location and (len(location)==5):
            thecommand = "(has %s location %s)" % (block1,location)

        
    if thecommand != "":
        print(thecommand)
        with open(outputfile, "a") as myfile:
            myfile.write(thecommand+"\n")

In [305]:
#os.remove("blockworld.txt") if os.path.exists("blockworld.txt") else None
#os.remove("goalworld.txt") if os.path.exists("goalworld.txt") else None

def readinfile(inputfile, outputfile):
    #os.remove(inputfile) if os.path.exists(inputfile) else None
    os.remove(outputfile) if os.path.exists(outputfile) else None

    commandFile = open(inputfile) 
    data = commandFile.read()# Use this to read file content as a stream: 
    commandFile.close()

    sentences = sent_tokenize(data)
    #print(sentences)
    parts_of_speech = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        temp = []
        for w in words:
            temp.append(w.lower())
        words = temp[0:-1]
        temp = []
        
        for w in words:
            if(w in nouns):
                temp.append((w, "NN"))
            elif(w in verbs_top):
                temp.append((w, "VT"))
            elif(w in verbs_below):
                temp.append((w, "VB"))
            elif(w in verbs_side):
                temp.append((w, "VS"))
            elif(w in variables_colors):
                temp.append((w, "VarC"))
            elif(w in variables_location):
                temp.append((w, "VarL"))
        tagged_words = temp
        #print(tagged_words)
        createBlockWorld(tagged_words, outputfile)
    
    #Part of Speech tagging
    #parts_of_speech.append(nltk.pos_tag(words))
    
#print("\nParts of Speech for each sentence")
#for p in parts_of_speech:
#    print(p)
    

    

In [306]:


#Read in initial file
readinfile("nlp_tests/start2.txt", "aStarSearch/nlp_outputs/start2.txt")

#Read in goal file
readinfile("nlp_tests/goal2.txt", "aStarSearch/nlp_outputs/goal2.txt")


tagged sentence
[('block1', 'NN'), ('blue', 'VarC')]
(has block1 color blue)
tagged sentence
[('block1', 'NN'), ('2', 'VarL'), ('3', 'VarL'), ('0', 'VarL')]
(has block1 location 2 3 0)
tagged sentence
[('block2', 'NN'), ('red', 'VarC')]
(has block2 color red)
tagged sentence
[('block2', 'NN'), ('2', 'VarL'), ('3', 'VarL'), ('1', 'VarL')]
(has block2 location 2 3 1)
tagged sentence
[('block3', 'NN'), ('yellow', 'VarC')]
(has block3 color yellow)
tagged sentence
[('block3', 'NN'), ('5', 'VarL'), ('1', 'VarL'), ('0', 'VarL')]
(has block3 location 5 1 0)
tagged sentence
[('block4', 'NN'), ('green', 'VarC')]
(has block4 color green)
tagged sentence
[('block4', 'NN'), ('8', 'VarL'), ('7', 'VarL'), ('0', 'VarL')]
(has block4 location 8 7 0)
tagged sentence
[('block5', 'NN'), ('cyan', 'VarC')]
(has block5 color cyan)
tagged sentence
[('block5', 'NN'), ('8', 'VarL'), ('7', 'VarL'), ('1', 'VarL')]
(has block5 location 8 7 1)
tagged sentence
[('block6', 'NN'), ('purple', 'VarC')]
(has block6 colo

In [311]:
#!{sys.executable} -m pip install pandas
#import pandas as pd
%run -i 'aStarSearch/aStarSearch.py' 'aStarSearch/nlp_outputs/start2.txt' 'aStarSearch/nlp_outputs/goal2.txt'

KeyboardInterrupt: 